## Importando Librerías

In [12]:
from wikiframe import Say, Extractor #Extrae los csv files
import numpy as np  #Libreria para trabajar con arrays
import pandas as pd #Libreria para trabajar con dataframes

import warnings
warnings.simplefilter("ignore")

#Regresores
from sklearn.linear_model import ElasticNet 
from sklearn.linear_model import HuberRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor


from sklearn.model_selection import RandomizedSearchCV,cross_val_score,KFold, GridSearchCV

from sklearn.pipeline import make_pipeline 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler,StandardScaler, OneHotEncoder,PowerTransformer, Normalizer
from sklearn.compose import ColumnTransformer
from mlxtend.regressor import StackingCVRegressor

## Importamos la métrica propuesta

Se utilizará para evaluar el desempeño del/los modelo/s la raíz cuadrada del error logarítmico medio, (RMSLE) por sus siglas en inglés.

$$ RMSLE=\sqrt{\frac{1}{n}\sum_{i=1}^n(\log (p_i + 1)-\log (a_i+1))^2}$$

In [13]:
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_log_error,  make_scorer
scoring=make_scorer(mean_squared_log_error, greater_is_better=False, squared=False)

## Variables Principales

Para el desarrollo del modelo , se tomó todas la columnas de la tabla de datos.

In [14]:
#Instanciar el objeto Extractor
extractor = Extractor('data')

#Extraer en df de ./data
df_dict = extractor.extract_from_csv()

#Crear dataframe con los datos
train = df_dict['house_train_raw'].drop(['Id'],axis=1)
test = df_dict['houses_test_raw'].drop(['Id'],axis=1)

In [15]:
train = pd.read_csv('https://raw.githubusercontent.com/MasamioNakada/Housing-Prices/main/data/house_train_raw.csv')
test = pd.read_csv('https://raw.githubusercontent.com/MasamioNakada/Housing-Prices/main/data/houses_test_raw.csv')

In [16]:
#Separamos en train y test
X = train.drop(['SalePrice'], axis=1)
y = train['SalePrice']

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
#Sepramos las varaiables categoricas y las numericas
categorical_cols = [x for x in X if X[x].dtype == "object"]
numerical_cols = [x for x in X if X[x].dtype == "int64" or train[x].dtype == "float64"]

## Transformación de Datos

- Para la variables categoricas , se realizó one-hot encoding, Cuando hay valores nulos simplemente a las columnas correspondientes será 0.
- Para las variables numéricas , si se encuentra un valor nulo, se le asignará la mediana de la columna correspondiente.

In [18]:
#Pipelines

#Pipeline para la variable categorica
from sklearn.preprocessing import MinMaxScaler


categorical = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), #Llena los NaN con el valor mas frecuentes
    ("oneHot", OneHotEncoder(handle_unknown="ignore")) #Codifica las variables categoricas
])

#Pipeline para la variable numerica
numerical = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")), #Llena los NaN con la mediana de la columna
    ("scaler", MinMaxScaler()) #Transforma los valores de la columna, se eligió PowerTransformer porque trata de hacer gausiana la distribucion de los valores
])

#Pipeline para las variables categoricas y numericas
preproces = ColumnTransformer(
    transformers=[
        ('cat', categorical, categorical_cols),
        ('num', numerical, numerical_cols)
])

## Modelo

Se decidió utilizar **StackingRegressor** para enriquecer el modelo con distintos regresores.
- ElasticNetRegressor : En el EDA se observó que habia columnas que estabam altamente correlacionas. Este regresor es la conjunción de RidgeRegressor y LassoRegressor que su objetivo es penalizar esas columnas que podrían o no aportar al modelo.
- HuberRegressor : En el EDA se observó que había outiliers , por lo que a este regresor no le impacta muchos estos datos atípicos.
- ADABoostRegressor : Este regresor es menos propenso al sobreajuste ya que los parámetros de entrada no se optimizan conjuntamente. 
- RandonForestRegressor : En el entrenamiento por individual, este regresor dió los mejores resultados.
- XGBoostRegressor : En el entrenamiento por individual, este regresor dió los mejores resultados

In [19]:
#Elegimos los mejores Regresores
huber_regressor = HuberRegressor()
elastic_net = ElasticNet()
randon_forest = RandomForestRegressor()
ada_boost = AdaBoostRegressor()
gradient_boost = GradientBoostingRegressor()
xg_boost = XGBRegressor()

In [20]:
#Utilizamos StackingCVRegressor para combinar los regresores y tner una mejor estimacion
model = StackingCVRegressor(
    regressors=[huber_regressor, elastic_net, randon_forest,ada_boost,xg_boost],
    meta_regressor=elastic_net
)

#Pipeline para el modelo general
pipe = Pipeline(steps=[
    ('  ', preproces),
    ('model', model)
])

In [ ]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
scoring(y_test, y_pred)

In [21]:
#Establecemos los parametros del modelo para que sean optimizados
grid =RandomizedSearchCV(
    pipe, 
    param_distributions={

        'model__xgbregressor__min_child_weight':np.arange(1,10,1),
        'model__xgbregressor__learning_rate':np.arange(0.01,1,0.01),
        'model__xgbregressor__n_estimators':np.arange(1100,2000,80),
        
        
        'model__elasticnet__alpha':np.arange(0.2,1,0.1),
        'model__elasticnet__l1_ratio':np.arange(0.2,1,0.1),

        
        'model__randomforestregressor__n_estimators': np.arange(100,2000,100),
        'model__randomforestregressor__max_features': ['sqrt', 'log2', None],
        'model__randomforestregressor__max_depth': [ 60, 70, 80, 90, 100,],
        'model__randomforestregressor__min_samples_split':  [2, 5, 10],
        'model__randomforestregressor__min_samples_leaf': [1, 2, 4],

    },
    cv=KFold(n_splits=5,shuffle=True), 
    n_jobs=-1, 
    verbose=1,
    scoring=scoring,
    n_iter=10
    )

In [ ]:
# Para saber los nombres de los paramatros que queremos optimizar en RandomizedSearchCV
for param in grid.get_params().keys():
    print(param)    

In [22]:
grid.fit(X_train, y_train)
print('RMLS: ',np.abs(grid.best_score_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
RMLS:  0.12476997462084531


In [23]:
Say(f'RMLS: {np.abs(grid.best_score_)}').cow_says_good()

 ___________________________ 
< RMLS: 0.12476997462084531 > 
 --------------------------- 
        \   ^__^ 
         \  (oo)\_______ 
            (__)\ good🥇 )\/\ 
                ||----w | 
                ||     || 


## Resultados

Se obtuvo una métrica del 0.13 gracias a elegir correctamente los hyperparámetros del Pipeline. 

Se procederá a predecir el dataset de Test

In [50]:
test.shape

(1459, 80)

In [24]:
#Obtenemos la prediccion 
y_pred = pd.DataFrame(grid.predict(test),columns=['pred'])
path_pred = './out/pred_test.csv'
y_pred.to_csv(path_pred) #-> Guardamos solo la columna predicion

Say(path_pred).cow_says_good()

test['SalePrice'] = y_pred
path_all = './out/houses_test_raw.csv'
test.to_csv(path_all,index=False)

Say(path_all).cow_says_good()

 _____________________ 
< ./out/pred_test.csv > 
 --------------------- 
        \   ^__^ 
         \  (oo)\_______ 
            (__)\ good🛐 )\/\ 
                ||----w | 
                ||     || 
 ___________________________ 
< ./out/houses_test_raw.csv > 
 --------------------------- 
        \   ^__^ 
         \  (oo)\_______ 
            (__)\ good🥇 )\/\ 
                ||----w | 
                ||     || 
